# 构建代理
LangChain 支持创建智能体，即使用大型语言模型作为推理引擎来决定采取哪些行动以及执行行动所需的输入。执行行动后，可以将结果反馈给大型语言模型，以判断是否需要更多行动，或者是否可以结束。这通常通过工具调用实现。

In [1]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite


   -------------------- ------------------- 2/4 [langgraph-checkpoint-sqlite]
   ---------------------------------------- 4/4 [langchain-tavily]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

 ········


In [3]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [4]:
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
search_results = search.invoke("北京的天气")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

{'query': '北京的天气', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'url': 'https://www.nmc.cn/publish/forecast/ABJ/beijing.html', 'title': '北京-天气预报 - 中央气象台', 'content': '北京 18.3℃ 西南风 微风 交通气象预报 森林火险预报 草原火险预报 台风海洋 台风快讯与报文 台风路径预报 台风公报 台风预警 海区预报 海事公报 海洋天气预报 近海海雾预报 海区风力预报 北太平洋分析与预报 全球热带气旋监测公报 台风命名 台风综合信息 全球预报 全球天气公报 全球热带气旋监测公报 WMO第XI海区海事天气公报 全球灾害性天气监测月报 全球雨雪落区预报 世界气象中心（北京）门户网 一带一路气象服务 亚洲沙尘暴预报专业气象中心 环境气象 雾预报 霾预报 沙尘天气预报 空气污染气象条件预报 环境气象公报 大气环境气象公报 农业气象 土壤水分监测 + 北京 北京 微风 19℃ 13℃ 微风 微风 15℃ 10℃ 微风 微风 14℃ 6℃ 微风 微风 13℃ 3℃ 微风 微风 11℃ 2℃ 微风 微风 10℃ 5℃ 微风 微风 13℃ 5℃ 微风 18.4℃ 18.9℃ 19.4℃ 16.2℃ 14.6℃ 14.4℃ 13.9℃ 13.5℃ 13.9℃ 14.9℃ 13.5℃ 11.9℃ 11.4℃ 11.1℃ 10.7℃ 10.8℃ 12℃ 13.8℃ 13.2℃ 11.8℃ 8.8℃ 7.2℃ 6.2℃ 7.4℃ 10.2℃ 11.2℃ 12.8℃ 9.7℃ 5.8℃ 4.8℃ 3.2℃ 8.1℃ 8.2℃ 8.5℃ 10.8℃ 6.1℃ 西南风 3.1℃ 2.4℃ 2.2℃ 6.8℃ 8.5℃ 8.8℃ 9.8℃ 6.6℃ 5.9℃ 10.8℃ 11.9℃ 12.8℃ 5.2℃', 'score': 0.78493005, 'raw_content': None}, {'url': 'https://weather.cma.cn/web/weather/', 'title': '中国气象局-天气预报-城市预报- 北京', 'content': '| 气温 | 24

In [5]:
import getpass
import os

if not os.environ.get("DEEPSEEK_API_KEY"):
  os.environ["DEEPSEEK_API_KEY"] = getpass.getpass("Enter API key for DeepSeek: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-chat", model_provider="deepseek")

Enter API key for DeepSeek:  ········


In [6]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! 👋 How can I help you today?'

In [7]:
model_with_tools = model.bind_tools(tools)

In [8]:
query = "Hi!"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: Hello! 👋 I'm here to help you with whatever you need. Whether you're looking for information, have questions, or need assistance with research, I'm ready to assist. What can I do for you today?

Tool calls: []


In [9]:
query = "查找上海崇宁的天气"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 我来帮您查找上海崇宁的天气信息。

Tool calls: [{'name': 'tavily_search', 'args': {'query': '上海崇宁天气预报', 'search_depth': 'basic'}, 'id': 'call_00_2wPyTiRPqbtPQZXa6SSkbHY9', 'type': 'tool_call'}]


我们可以看到现在没有文本内容，但有一个工具调用！它要求我们调用 Tavily 搜索工具。

这还没有实际调用该工具——它只是在告诉我们应该调用。为了实际调用它，我们需要创建我们的智能体。

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

现在我们已经定义了工具和大型语言模型，我们可以创建智能体了。我们将使用LangGraph来构建智能体。目前，我们正在使用一个高级接口来构建智能体，但 LangGraph 的优点在于，这个高级接口由一个低级别、高度可控的 API 支持，以防你想要修改智能体逻辑。

现在，我们可以使用大型语言模型和工具初始化智能体。

请注意，我们传入的是model，而不是model_with_tools。这是因为create_react_agent将在底层为我们调用.bind_tools。

In [11]:
input_message = {"role": "user", "content": "Hi!"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Hi!
================================== Ai Message ==================================

Hello! 👋 I'm here to help you with any questions or tasks you might have. Whether you need information, research, analysis, or just want to chat, I'm ready to assist! What can I do for you today?


In [12]:
input_message = {"role": "user", "content": "查找上海闵行的天气"}
response = agent_executor.invoke({"messages": [input_message]})

for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

查找上海闵行的天气
================================== Ai Message ==================================

我来为您查找上海闵行的天气信息。
Tool Calls:
  tavily_search (call_00_JMbrDndcJr0bI1GnN7KFp6i3)
 Call ID: call_00_JMbrDndcJr0bI1GnN7KFp6i3
  Args:
    query: 上海闵行天气 今天 温度 降雨
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "上海闵行天气 今天 温度 降雨", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://tianqi.moji.com/weather/china/shanghai/minhang-district", "title": "闵行区天气预报_天气查询 - 墨迹天气", "content": "闵行区今天实况：19度多云，湿度：74%，北风：3级。白天：24度,阴。 夜间：小雨，17度，天气较热，墨迹天气建议您选择短袖上衣加七分裤的搭配，针织衫是进出空调房的必备单品。", "score": 0.86108357, "raw_content": null}, {"url": "https://www.accuweather.com/zh/cn/minhang-district/60811/weather-forecast/60811", "title": "閔行區, 上海市, 中國三日天氣預報 - AccuWeather", "content": "今天. 10/18. 75° 66°. 零星有雨和雷暴. 局部多雲轉多雲 ; 周日

In [14]:
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"): # 设置流式传输，使得信息能够逐步展示出来
    step["messages"][-1].pretty_print()

================================ Human Message =================================

查找上海闵行的天气
================================== Ai Message ==================================

我来帮您查找上海闵行的天气信息。
Tool Calls:
  tavily_search (call_00_hC8rNQFV9S5B6uLnp9cj0Ghp)
 Call ID: call_00_hC8rNQFV9S5B6uLnp9cj0Ghp
  Args:
    query: 上海闵行天气
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "上海闵行天气", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://tianqi.moji.com/weather/china/shanghai/minhang-district", "title": "闵行区天气预报_天气查询 - 墨迹天气", "content": "闵行区今天实况：19度多云，湿度：74%，北风：3级。白天：24度,阴。 夜间：小雨，17度，天气较热，墨迹天气建议您选择短袖上衣加七分裤的搭配，针织衫是进出空调房的必备单品。", "score": 0.852752, "raw_content": null}, {"url": "https://www.weather.com.cn/weather/101020200.shtml", "title": "闵行天气预报,闵行7天天气预报,闵行15天天气预报,闵行天气查询", "content": "更多 :   北京 上海 成都 杭州 南京 天津 深圳 重庆 西安 广州 青岛 武汉 全国> 上海 > 闵行 26/*22℃* *3-4级转<3级* 29/*21℃* *<3级* 

In [15]:
for step, metadata in agent_executor.stream( # 设置流式传输令牌也可以达到效果
    {"messages": [input_message]}, stream_mode="messages"
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

我来|帮|您|查找|上海|闵|行的|天气|信息|。|根据|搜索结果|，|我|为您|整理了|上海|闵|行|区的|天气|信息|：

|**|当前|天气|情况|：|**
|-| |天气|：|阴|天|
|-| |温度|：|18|°|C| /| |22|°|C|
|-| |风向|风力|：|北|风|3|级|

|**|未来|几天|天气预报|：|**
|-| |明天|：|阴|天|，|15|°|C| /| |19|°|C|，|东北|风|1|级|
|-| |后天|：|15|°|C| /| |19|°|C|，|北|风|1|级|

|**|生活|指数|：|**
|-| |感冒|指数|：|较|易|发|
|-| |运动|指数|：|较|不宜|
|-| |过敏|指数|：|较|易|发|
|-| |洗|车|指数|：|不宜|

|**|温馨提示|：|**|
|目前|闵|行|区|天气|以|阴|天|为主|，|温度|适中|但|早晚|温差|较大|，|建议|您|适当|添加|衣物|，|注意|预防|感冒|。|由于|天气|状况|不佳|，|不太|适合|户外|运动|。

|数据|更新时间|：|今天|22|:|54|

添加记忆

In [16]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [18]:
input_message = {"role": "user", "content": "Hi, I'm Bob!"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Hi, I'm Bob!
================================== Ai Message ==================================

Hi Bob! Nice to meet you! I'm here to help you with any questions or tasks you might have. Whether you need information, research assistance, or just want to chat about something you're curious about, I'm ready to help.

What can I do for you today?


In [19]:
input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

Your name is Bob! You introduced yourself at the beginning of our conversation. It's nice to meet you, Bob! 😊

Is there anything specific you'd like to talk about or any questions I can help you with?


In [20]:
# 更改线程，这样就不会记忆另一个线程的内容
config = {"configurable": {"thread_id": "xyz123"}}

input_message = {"role": "user", "content": "What's my name?"}
for step in agent_executor.stream(
    {"messages": [input_message]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What's my name?
================================== Ai Message ==================================

I don't have access to personal information about you, including your name. To help me address you properly, you could tell me your name directly, or if you're looking for information about yourself from other sources, you might need to provide more context about what specific information you're seeking.
